In [90]:
from utils.parliament_utils import *
import numpy as np
import pickle
from tqdm.notebook import tqdm
tqdm.pandas()

## Collection

In [22]:
url = 'ftp://oda.ft.dk/ODAXML/Referat/samling'
out_path = 'data/raw/parliament/legislative_sessions_dir.xml'
save_xml_from_ftp(url, out_path)

In [23]:
with open(out_path, 'r') as f:
    sessions_dir = f.readlines()

In [24]:
parliamentary_sessions = [line.replace('\n', '').split('>          ')[-1] for line in sessions_dir]

In [25]:
parliamentary_sessions

['20091',
 '20101',
 '20102',
 '20111',
 '20121',
 '20131',
 '20141',
 '20142',
 '20151',
 '20161',
 '20171',
 '20181',
 '20182',
 '20191',
 '20201',
 '20211',
 '20221']

In [26]:
filenames = list_meetings_in_sessions(parliamentary_sessions)

  0%|          | 0/17 [00:00<?, ?it/s]

In [29]:
filenames[:3]

['data/raw/parliament/session_docs/session_20091.xml',
 'data/raw/parliament/session_docs/session_20101.xml',
 'data/raw/parliament/session_docs/session_20102.xml']

In [30]:
all_transcript_filenames = extract_transcript_filenames(filenames)

  0%|          | 0/17 [00:00<?, ?it/s]

In [33]:
all_transcript_filenames['20221']

['20221_M1_helemoedet.xml',
 '20221_M2_helemoedet.xml',
 '20221_M3_helemoedet.xml']

In [34]:
subset_list = [
     '20211',
     '20221',
]

In [35]:
subset_dict = {k: all_transcript_filenames[k] for k in subset_list}

In [37]:
save_transcripts_from_ftp(subset_dict)

  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

## Parsing XML

In [ ]:
xml_str = xml_to_str(xml_file)

In [30]:
xml_dicts = xml_str_to_rows(xml_str)

In [43]:
pd.DataFrame(xml_dicts)['text'][4]

'Jeg tager ordet, også, fordi vi, som der står i vores betænkningsbidrag, faktisk var lidt overraskede over, at man blander tingene sammen, for det synes vi man gør her. Vi er meget indstillet på at hjælpe, hvor vi kan, i forhold til ukrainerne. Vi er gået så vidt, at vi også stemmer for en særlov. Men vi var meget undrende, da det gik op for os – og i forhandlingen bagefter virkede det, som om også integrationsministeren var lidt overrasket over, at afghanere også kommer ind under den her ordning. Det er jo placeret i to ministerier, altså også i Beskæftigelsesministeriet, så måske har der ikke været en ordentlig koordinering imellem ministerierne. Derfor vil vi bede om, at forslaget bliver delt op, sådan at man kan stemme individuelt, og så vi kan give vores tilslutning til Ukraine, men ikke til de afghanske tolke.'

In [39]:
all_xml_to_df('data/raw/parliament/xml_transcripts', 'data/clean/parliament/transcripts_v3.csv')

  0%|          | 0/1505 [00:00<?, ?it/s]

Whoops, IsADirectoryError encountered in data/raw/parliament/xml_transcripts/.ipynb_checkpoints!


In [67]:
df_politicians = pd.read_csv('data/clean/parliament/transcripts_v3.csv')

In [68]:
df_politicians

,first_name,last_name,group_name,role,start_time,end_time,text
0,Niels Helveg,Petersen,RV,aldersformanden,2009-10-06T12:00:32,2009-10-06T12:02:08,Mødet er åbnet. I henhold til grundloven er Fo...
1,Thor,Pedersen,V,formand,2009-10-06T12:02:08,2009-10-06T12:04:01,"Jeg vil gerne takke for den tillid, som Tinget..."
2,Lars Løkke,Rasmussen,NaN,minister,2009-10-06T12:04:01,2009-10-06T12:50:39,Danmark er et godt land at leve i. Vi har bygg...
3,Thor,Pedersen,V,formand,2009-10-06T12:50:39,2009-10-06T12:51:10,Tak til statsministeren. Jeg kan oplyse Folket...
4,MødeSlut,MødeSlut,MødeSlut,MødeSlut,2009-10-06T12:51:10,NaN,Mødet er hævet. (Kl. 12:51).
...,...,...,...,...,...,...,...
635791,Mette,Frederiksen,NaN,minister,2022-10-06T23:19:22,2022-10-06T23:19:57,"Jeg er ikke i tvivl om, at vi har behov for at..."
635792,Henrik Dam,Kristensen,S,formand,2022-10-06T23:19:57,2022-10-06T23:21:46,Nu skal statsministeren have fri. Det var det ...
635793,Henrik Dam,Kristensen,S,formand,2022-10-06T23:21:46,2022-10-06T23:22:30,Der stemmes først om forslag til vedtagelse nr...
635794,Henrik Dam,Kristensen,S,formand,2022-10-06T23:22:30,2022-10-06T23:22:50,Mødet er slut. God valgkamp.


In [69]:
635796 - 633644

2152

In [70]:
pol_filtered = df_politicians.loc[
    (df_politicians['first_name'] != 'MødeSlut') &
    (df_politicians['role'] != 'Pause') &
    (df_politicians['role'] != 'formand') &
    (df_politicians['role'] != 'aldersformanden')
].dropna(
    subset=['first_name', 'last_name', 'group_name', 'role'], # these are questions that have been rescinded
    how='all'
).copy()

In [71]:
pol_filtered

,first_name,last_name,group_name,role,start_time,end_time,text
2,Lars Løkke,Rasmussen,NaN,minister,2009-10-06T12:04:01,2009-10-06T12:50:39,Danmark er et godt land at leve i. Vi har bygg...
8,Bjarne,Laustsen,S,medlem,2009-10-07T13:04:25,2009-10-07T13:06:31,"Tak, hr. formand. Den sag, jeg vil bringe på b..."
10,Eva Kjer,Hansen,NaN,minister,2009-10-07T13:06:41,2009-10-07T13:08:09,Jeg må starte med at rette hr. Bjarne Laustsen...
12,Bjarne,Laustsen,S,medlem,2009-10-07T13:08:10,2009-10-07T13:09:17,"Tak. Der er ikke så meget nyt under solen, mås..."
14,Eva Kjer,Hansen,NaN,minister,2009-10-07T13:09:19,2009-10-07T13:10:17,"Jeg synes, at det sidste, der blev sagt fra hr..."
...,...,...,...,...,...,...,...
635783,Mette,Frederiksen,NaN,minister,2022-10-06T23:16:13,2022-10-06T23:16:46,Vi er jo meget langt nede nu i noget materiale...
635785,Anne Valentina,Berthelsen,SF,medlem,2022-10-06T23:16:47,2022-10-06T23:17:56,Tak til statsministeren for talen og for samar...
635787,Mette,Frederiksen,NaN,minister,2022-10-06T23:17:57,2022-10-06T23:18:43,"Altså, hvis vi både kigger på Østersøregionen ..."
635789,Anne Valentina,Berthelsen,SF,medlem,2022-10-06T23:18:44,2022-10-06T23:19:20,"Jo, men kunne man ikke forestille sig, at vi, ..."


In [72]:
# Create full name from first + last name
pol_filtered['full_name'] = pol_filtered['first_name'] + ' ' + pol_filtered['last_name']

In [73]:
pol_filtered['full_name'].value_counts(dropna = False)

Ole Birk Olesen          4071
Finn Sørensen            3359
Mette Frederiksen        3325
Christian Juhl           3270
Inger Støjberg           3035
                         ... 
Jens Arne Hedegaard         1
Niels Høiby                 1
Peder Christensen           1
Sisse Marie Welling         1
Anders G. Christensen       1
Name: full_name, Length: 484, dtype: int64

In [74]:
pol_filtered.loc[pol_filtered.isna().any(axis=1), 'full_name'].value_counts()

Mette Frederiksen           2736
Inger Støjberg              2348
Morten Bødskov              1900
Nick Hækkerup               1771
Lars Løkke Rasmussen        1726
                            ... 
Christian Rabjerg Madsen      54
Jonas Dahl                    35
Søren Gade                    33
Kaare Dybvad                  32
Connie Hedegaard              23
Name: full_name, Length: 98, dtype: int64

In [75]:
# It would seem that we have to impute party affiliation for ministers and other special cases,
# since title is included instead of party info when, for instance, the speaker is Minister of X.
#
# Since a given cabinet may consist of more than one party, the most fail-safe procedure seems to
# be imputing current party membership by locating the most recent extant record of membership
# prior to appointment as minister.

In [76]:
def impute_party_membership(df, politician_list):
    '''
    TODO...
    '''
    
    new_df = df.copy()
    
    for politician in tqdm(politician_list):
        
        # Get party affiliation (potentially multiple parties)
        party_affiliations = new_df.loc[
            (new_df['full_name'] == politician) &
            new_df['group_name'].notna(),
            'group_name'
        ].drop_duplicates().tolist()
        
        try:
            (party_imputation,) = party_affiliations # unpack party list if it only contains 1 element

        except ValueError:
            party_imputation = ','.join(party_affiliations) # return entire list as str, if more than one membership
            print(f'{politician} has been a member of multiple parties,\nor party affiliation could not be determined: {party_imputation}.\n')

        # Write party membership where missing
        new_df.loc[
            (new_df['full_name'] == politician) &
            new_df['group_name'].isna(),
            ['group_name']
        ] = party_imputation
    
    return new_df

In [77]:
unknown_party_politicians = (
    pol_filtered
        .loc[pol_filtered['group_name'].isna(), 'full_name']
        .drop_duplicates()
        .tolist()
)

In [81]:
unknown_party_politicians[:5]

['Lars Løkke Rasmussen',
 'Eva Kjer Hansen',
 'Claus Hjort Frederiksen',
 'Troels Lund Poulsen',
 'Karen Ellemann']

In [82]:
pol_imputed = impute_party_membership(pol_filtered, unknown_party_politicians)

  0%|          | 0/98 [00:00<?, ?it/s]

Lars Løkke Rasmussen has been a member of multiple parties,
or party affiliation could not be determined: V,UFG,M.

Inger Støjberg has been a member of multiple parties,
or party affiliation could not be determined: V,UFG.

Connie Hedegaard has been a member of multiple parties,
or party affiliation could not be determined: .

Charlotte Sahl-Madsen has been a member of multiple parties,
or party affiliation could not be determined: .

Thor Möger Pedersen has been a member of multiple parties,
or party affiliation could not be determined: .

Astrid Krag has been a member of multiple parties,
or party affiliation could not be determined: SF,S.

Uffe Elbæk has been a member of multiple parties,
or party affiliation could not be determined: RV,UFG,ALT,FG.

Ida Auken has been a member of multiple parties,
or party affiliation could not be determined: SF,RV,S.

Jørn Neergaard Larsen has been a member of multiple parties,
or party affiliation could not be determined: .

Simon Emil Ammitzbøll-

In [83]:
(
    pol_imputed
        .loc[pol_imputed['group_name'] == '', 'full_name']
        .drop_duplicates()
        .tolist()
)

['Connie Hedegaard',
 'Charlotte Sahl-Madsen',
 'Thor Möger Pedersen',
 'Jørn Neergaard Larsen',
 'Peter Hummelgaard',
 'Kaare Dybvad Bek',
 'Joy Mogensen']

In [84]:
manual_party_map = {
    'Connie Hedegaard': 'KF',
    'Charlotte Sahl-Madsen': 'KF',
    'Thor Möger Pedersen': 'SF',
    'Jørn Neergaard Larsen': 'V',
    'Peter Hummelgaard': 'S',
    'Kaare Dybvad Bek': 'S',
    'Joy Mogensen': 'S'
}

In [85]:
# Replace empty strings with NaN
pol_imputed['group_name'] = pol_imputed['group_name'].replace('', np.nan)

# Fillna using manual map
pol_imputed['group_name'] = pol_imputed['group_name'].fillna(pol_imputed['full_name'].map(manual_party_map))

In [86]:
pol_imputed['year'] = pol_imputed['start_time'].str.extract(r'(\d\d\d\d)').astype(int)
pol_imputed['day'] = pol_imputed['start_time'].str.extract(r'\d\d\d\d-(\d\d-\d\d)T\d\d:\d\d:\d\d')
pol_imputed['time'] = pol_imputed['start_time'].str.extract(r'\d\d\d\d-\d\d-\d\dT(\d\d:\d\d:\d\d)')
pol_imputed['duration'] = pol_imputed.progress_apply(lambda row: parse_datetime(row['end_time']) - parse_datetime(row['start_time']), axis = 1)

  0%|          | 0/302588 [00:00<?, ?it/s]

In [87]:
col_order = [
    'first_name',
    'last_name',
    'full_name',
    'group_name',
    'role',
    'text',
    'year',
    'day',
    'time',
    'start_time',
    'end_time',
    'duration'
]

In [88]:
pol_final = pol_imputed.reset_index(drop = True)[col_order]

In [91]:
#pol_final.to_pickle('data/clean/parliament/transcripts_imputed_v2.p')

In [92]:
with open('data/clean/parliament/transcripts_imputed_v2.p', 'rb') as p:
    test = pickle.load(p)

In [93]:
test.loc[test['year'] >= 2018].shape

(113699, 12)

In [94]:
test.shape

(302588, 12)

In [96]:
test.loc[test['year'] > 2021]

,first_name,last_name,full_name,group_name,role,text,year,day,time,start_time,end_time,duration
286959,Troels,Ravn,Troels Ravn,S,medlem,Tak for ordet. I Socialdemokratiet er vi glade...,2022,01-11,13:02:59,2022-01-11T13:02:59,2022-01-11T13:04:35,0 days 00:01:36
286960,Louise Schack,Elholm,Louise Schack Elholm,V,medlem,Tak. Det her er det årlige lovforslag om at sæ...,2022,01-11,13:04:58,2022-01-11T13:04:58,2022-01-11T13:05:29,0 days 00:00:31
286961,Dennis,Flydtkjær,Dennis Flydtkjær,DF,medlem,Tak. Der er jo tit genfremsættelser af forslag...,2022,01-11,13:05:44,2022-01-11T13:05:44,2022-01-11T13:06:31,0 days 00:00:47
286962,Jan Bjergskov,Larsen,Jan Bjergskov Larsen,SF,medlem,Mange tak for ordet. Jeg er her som vikar for ...,2022,01-11,13:06:47,2022-01-11T13:06:47,2022-01-11T13:08:08,0 days 00:01:21
286963,Kathrine,Olldag,Kathrine Olldag,RV,medlem,"Tusind tak, formand. Det skal også være en kor...",2022,01-11,13:08:23,2022-01-11T13:08:23,2022-01-11T13:08:52,0 days 00:00:29
...,...,...,...,...,...,...,...,...,...,...,...,...
302583,Mette,Frederiksen,Mette Frederiksen,S,minister,Vi er jo meget langt nede nu i noget materiale...,2022,10-06,23:16:13,2022-10-06T23:16:13,2022-10-06T23:16:46,0 days 00:00:33
302584,Anne Valentina,Berthelsen,Anne Valentina Berthelsen,SF,medlem,Tak til statsministeren for talen og for samar...,2022,10-06,23:16:47,2022-10-06T23:16:47,2022-10-06T23:17:56,0 days 00:01:09
302585,Mette,Frederiksen,Mette Frederiksen,S,minister,"Altså, hvis vi både kigger på Østersøregionen ...",2022,10-06,23:17:57,2022-10-06T23:17:57,2022-10-06T23:18:43,0 days 00:00:46
302586,Anne Valentina,Berthelsen,Anne Valentina Berthelsen,SF,medlem,"Jo, men kunne man ikke forestille sig, at vi, ...",2022,10-06,23:18:44,2022-10-06T23:18:44,2022-10-06T23:19:20,0 days 00:00:36
